In [785]:
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
# import seaborn as sns


In [786]:
df= pd.read_csv('./dataset.csv')
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# PRE PROCESSING

In [787]:
from time import time


#convert str to date
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])
df.count()

InvoiceNo      541909
StockCode      541909
Description    540455
Quantity       541909
InvoiceDate    541909
UnitPrice      541909
CustomerID     406829
Country        541909
dtype: int64

In [788]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [789]:
df=df[df['CustomerID'].notna()]

In [790]:

df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [791]:
""
from datetime import datetime
from random import triangular


start_date=pd.Timestamp(2010,12,1)
end_date=pd.Timestamp(2011,8,31)
train_set=df[df['InvoiceDate']>=start_date]
train_set=train_set[train_set['InvoiceDate']<=end_date]

In [792]:
train_set.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [793]:
start_date=pd.Timestamp(2011,9,1)
end_date=pd.Timestamp(2011,12,31)
churn_set=df[df['InvoiceDate']>=start_date]
churn_set=churn_set[churn_set['InvoiceDate']<=end_date]
churn_set.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
320705,565080,20677,PINK POLKADOT BOWL,8,2011-09-01 08:25:00,1.25,13509.0,United Kingdom
320706,565080,22128,PARTY CONES CANDY ASSORTED,24,2011-09-01 08:25:00,1.25,13509.0,United Kingdom
320708,565082,22423,REGENCY CAKESTAND 3 TIER,2,2011-09-01 09:15:00,12.75,13305.0,United Kingdom
320709,565082,15060B,FAIRY CAKE DESIGN UMBRELLA,8,2011-09-01 09:15:00,3.75,13305.0,United Kingdom
320710,565082,23245,SET OF 3 REGENCY CAKE TINS,4,2011-09-01 09:15:00,4.95,13305.0,United Kingdom


In [794]:
train_unique=train_set['CustomerID'].unique()
churn_unique=churn_set['CustomerID'].unique()

ischurnlis=[]
for id in train_unique:
    if id in churn_unique:
        ischurnlis.append(id)


In [795]:
train_set=train_set.assign(isChurn=[1 if x in ischurnlis else 0 for x in train_set['CustomerID']])

In [796]:
train_set.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,isChurn
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0


In [797]:
train_set.count()

InvoiceNo      231183
StockCode      231183
Description    231183
Quantity       231183
InvoiceDate    231183
UnitPrice      231183
CustomerID     231183
Country        231183
isChurn        231183
dtype: int64

In [798]:
#taking care of country
countries_dic={}

countries=train_set['Country'].unique()
for i,y in enumerate(countries):
    countries_dic[y]=i
countries_dic


{'United Kingdom': 0,
 'France': 1,
 'Australia': 2,
 'Netherlands': 3,
 'Germany': 4,
 'Norway': 5,
 'EIRE': 6,
 'Switzerland': 7,
 'Spain': 8,
 'Poland': 9,
 'Portugal': 10,
 'Italy': 11,
 'Belgium': 12,
 'Lithuania': 13,
 'Japan': 14,
 'Iceland': 15,
 'Channel Islands': 16,
 'Denmark': 17,
 'Cyprus': 18,
 'Sweden': 19,
 'Austria': 20,
 'Israel': 21,
 'Finland': 22,
 'Greece': 23,
 'Singapore': 24,
 'Lebanon': 25,
 'United Arab Emirates': 26,
 'Saudi Arabia': 27,
 'Czech Republic': 28,
 'Canada': 29,
 'Unspecified': 30,
 'Brazil': 31,
 'USA': 32,
 'European Community': 33,
 'Bahrain': 34,
 'Malta': 35}

In [799]:
train_set['Country']=[countries_dic[x] for x in train_set['Country']]



In [800]:
train_set['Country'].unique()
#we assign country codes instead of country names

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35])

In [801]:

train_set.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,isChurn
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,0,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,0,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,0,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,0,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,0,0


In [802]:
train_set.count()

InvoiceNo      231183
StockCode      231183
Description    231183
Quantity       231183
InvoiceDate    231183
UnitPrice      231183
CustomerID     231183
Country        231183
isChurn        231183
dtype: int64

In [803]:
train_set.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'isChurn'],
      dtype='object')

In [804]:
len(train_set['StockCode'].unique())

3412

In [805]:
train_set=train_set.drop(['Description'],axis=1)

In [806]:
#stock code has some letters
#lets remove them and use it as a feature

train_set['StockCode']=train_set['StockCode'].str.replace('\D', '')

    

/home/ayushsingh/anaconda3/envs/ai_ml_nn/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [807]:
train_set=train_set[train_set['StockCode']!='']

In [808]:
train_set['StockCode']=train_set['StockCode'].astype(int)

In [809]:
train_set

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,isChurn
0,536365,85123,6,2010-12-01 08:26:00,2.55,17850.0,0,0
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850.0,0,0
2,536365,84406,8,2010-12-01 08:26:00,2.75,17850.0,0,0
3,536365,84029,6,2010-12-01 08:26:00,3.39,17850.0,0,0
4,536365,84029,6,2010-12-01 08:26:00,3.39,17850.0,0,0
...,...,...,...,...,...,...,...,...
319395,564852,82552,10,2011-08-30 17:23:00,1.45,14976.0,0,1
319396,564852,21756,3,2011-08-30 17:23:00,5.95,14976.0,0,1
319397,564852,21908,7,2011-08-30 17:23:00,2.10,14976.0,0,1
319398,564852,22116,10,2011-08-30 17:23:00,0.79,14976.0,0,1


In [810]:
train_set['CustomerID']=train_set['CustomerID'].astype(int)

# Dropping invoice no
train_set=train_set.drop(['InvoiceNo'],axis=1)
train_set=train_set.drop(['InvoiceDate'],axis=1)

In [811]:
train_set.columns

Index(['StockCode', 'Quantity', 'UnitPrice', 'CustomerID', 'Country',
       'isChurn'],
      dtype='object')

In [812]:
#normalising train set
from hmac import trans_36
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler1=StandardScaler()
scaler2=MinMaxScaler()

train_set['StockCode']=scaler2.fit_transform(train_set[['StockCode']])
train_set['CustomerID']=scaler2.fit_transform(train_set[['CustomerID']])
train_set['Country']=scaler2.fit_transform(train_set[['Country']])

train_set['Quantity']=scaler1.fit_transform(train_set[['Quantity']])
train_set['UnitPrice']=scaler1.fit_transform(train_set[['UnitPrice']])


In [813]:
y=train_set['isChurn']
x=train_set.drop(['isChurn'],axis=1)

In [814]:
#dividing data set into train test and dev
#80/20
X_train,X_test,y_train,y_test = sk.model_selection.train_test_split(x,y,test_size=0.2,random_state=1)

In [815]:
#80/20
X_train,X_val,y_train,y_val = sk.model_selection.train_test_split(X_train,y_train,test_size=0.2,random_state=1)

In [816]:
#decision Tree

from sklearn.tree import DecisionTreeClassifier

model1=DecisionTreeClassifier()
model1.fit(X_train,y_train)

DecisionTreeClassifier()

In [817]:
y_pred = model1.predict(X_test)

In [818]:
#evaluating decision tree
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 8536   264]
 [  209 37017]]
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      8800
           1       0.99      0.99      0.99     37226

    accuracy                           0.99     46026
   macro avg       0.98      0.98      0.98     46026
weighted avg       0.99      0.99      0.99     46026



In [819]:
#SVM
from sklearn.svm import LinearSVC

model2=LinearSVC()
model2.fit(X_train,y_train)

/home/ayushsingh/anaconda3/envs/ai_ml_nn/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


LinearSVC()

In [820]:
y_pred = model2.predict(X_test)

In [821]:
#evaluating SVM
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[   24  8776]
 [   17 37209]]
              precision    recall  f1-score   support

           0       0.59      0.00      0.01      8800
           1       0.81      1.00      0.89     37226

    accuracy                           0.81     46026
   macro avg       0.70      0.50      0.45     46026
weighted avg       0.77      0.81      0.72     46026



In [822]:
#ANN
from sklearn.neural_network import MLPClassifier

model3=MLPClassifier()
model3.fit(X_train,y_train)

MLPClassifier()

In [823]:
y_pred = model3.predict(X_test)

In [824]:
#evaluating neural network
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[  141  8659]
 [   49 37177]]
              precision    recall  f1-score   support

           0       0.74      0.02      0.03      8800
           1       0.81      1.00      0.90     37226

    accuracy                           0.81     46026
   macro avg       0.78      0.51      0.46     46026
weighted avg       0.80      0.81      0.73     46026

